# **TRAINING PROCESS**

<BR>

We will train using **three** different modeling approaches. All of their specific implementations are viewable at [src/classes](src/classes)

1. MultipleRegressor
2. ParzenWindow
3. VGG inspired Neural Network

All these models will be **compatible** with popular **Machine Learning** libraries `sklearn` or `PyTorch` through the extension of their respective `BaseModel` classes. 

The most resource-heavy models do not work properly within this `notebook` environments. We aim to explain the code and the training decisions here, but when its time to run the training cycles, we **strongly** suggest that you use the scripts instead.

<br>
<br>

# Table of Contents
- [**TRAINING PROCESS**](#training-process)

  - [**MultipleRegressionModel.py**](#multipleregressionmodel.py)  
    - [*Qualities of the MultipleRegressionModel.py*](#qualities-of-the-multipleregressionmodel.py)  


  - [**ParzenWindowModel.py**](#parzenwindowmodel.py)  

    - [*Qualities of the ParzenWindowModel.py*](#qualities-of-the-parzenwindowmodel.py)  

  - [**VGGUdeaSpectral.py**](#vggudeaspectral.py)  
    - [*Qualities of the VGGUdeaSpectral.py*](#qualities-of-the-vggudeaspectral.py)  
    


  - [**CONCLUSIONS DERIVED FROM THE RESULTS OF THE MODELS**](#conclusions-derived-from-the-results-of-the-models)  

  - [FUTURE WORK AND USES FOR THE MODEL](#future-work-and-uses-for-the-model)  

    - [Future Work](#future-work)  

    - [**USES FOR THE MODEL**](#uses-for-the-model)  
    
      - [**REFERENCES**](#references)  


<BR>


---

<br>

## **MultipleRegressionModel.py**

<br>

A multiple regressor  is a model  that is used to predict the outcome of a dependent variable based on the values of multiple independent variables. This is based on the assumption that there is a linear relationship between the independent variables and the dependent variable.

However, there are several reasons why a multiple regressor may not be the best option for a regression problem involving satellite images, we list here the ones that we consider most relevant:

<br>

1. **Non-linearity:** The relationship between the features in satellite images (such as pixel values) and the outcome variable is often complex and non-linear. Multiple regressors assume a linear relationship, which may not be sufficient to capture the patterns in satellite imagery.

2. **Spatial Autocorrelation:** Pixels in satellite images are often spatially correlated with nearby pixels, violating the multiple regression assumption that observations are independent of each other.

3. **Spectral Autocorrelation:** There is often a high correlation between different spectral bands in satellite images, which can lead to multicollinearity problems in multiple regression models.


The images need to be **flattenned** to be fed into the model.

<br>

Nonetheless, we implement it here as an academic exercise.

<br>

### *Qualities of the MultipleRegressionModel.py*

- **Model Definition**: Available at [src/classes/MultipleRegressionModel.py](src/classes/MultipleRegressionModel.py)

- **Training Cycle**: Available at [src/generate_models/train_MultipleRegression.py](src/generate_models/train_MultipleRegression.py)

- **Compatible with**: sklearn **[BaseEstimator](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html)** and **[RegressorMixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.RegressorMixin.html)**

- **Hyperparameter Optimization SearchSpace**: 

    ```python
    param_grid = {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'positive': [True, False]
    }
    ```
- **Model Optimization Utility**: `GridSearchCV` with scoring to *minimize MAE* and Shuffled Kfolds with `cv=3` and a 20% val split. The `RMSE` is used as a complementary metric.

    ```python
    model = GridSearchCV(
                        estimator=MultipleRegressionModel(),
                        param_grid=param_grid,
                        n_jobs=2,            # really unstable. move back to 1 if crashing. dont use -1
                        scoring='neg_mean_absolute_error',  #  minimize MAE
                        cv=3,                
                        verbose=3    
                    )
    ```



<br>
<br>
<br>

The code provided below is a slight modification of the one available on the scripts (we changed the import structure to match the package structure)




In [4]:
## CLASS DEFINITION

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression

class MultipleRegressionModel(BaseEstimator, RegressorMixin):
    def __init__(self, fit_intercept=True, copy_X=True, positive=True):
        """
        Hyperparameters:
        - fit_intercept: bool, default=True
            Whether to calculate the intercept for this model.
        - copy_X: bool, default=True
            If True, X will be copied; else, it may be overwritten.
        - positive: bool, default=True
            When set to True, forces the coefficients to be positive.
        """
        
        self.fit_intercept = fit_intercept
        self.copy_X = copy_X
        self.positive = positive
        
        self.model = LinearRegression(
            fit_intercept=self.fit_intercept, 
            copy_X=self.copy_X,
            n_jobs=-1,  # Always use the maximum number of jobs
            positive=self.positive
        )
        
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        return self.model.predict(X)


In [5]:
## training cycle

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import os
import json
import joblib
import numpy as np
import time
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


# Importing the custom model class
from src.classes.MultipleRegressionModel import MultipleRegressionModel

# Utility functions
from src.utils import *

def main():
    # Load the data
    X = torch.load('X_tensor.pth').numpy()
    y = torch.load('y_tensor.pth').numpy()
    
    # Flatten the images
    X = X.reshape(X.shape[0], -1)
    
    # Initialize K-Fold cross-validation
    kf = KFold(n_splits=2, shuffle=True, random_state=None)



    param_grid = {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'positive': [True, False]
    }

    # Initialize the model with hyperparameter grid
    model = GridSearchCV(
                        estimator=MultipleRegressionModel(),
                        param_grid=param_grid,
                        n_jobs=2,            # really unstable. move back to 1 if crashing. dont use -1
                        scoring='neg_mean_absolute_error',  #  minimize MAE
                        cv=3,                
                        verbose=3    
                    )
    
    # Initialize variables for cross-validation
    fold_mae_losses = []
    fold_rmse_losses = []
    
    # Create a folder for this specific model training
    model_type = "MultipleRegression"
    model_folder = generate_unique_folder(str(model_type))
    print(f"Folder {model_folder[0]} created at {model_folder[1]}")
    
    # Cross-validation loop
    for fold, (train_index, val_index) in enumerate(tqdm(kf.split(X), desc='KFold Progress')):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        # Train the model on the training set
        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()
        
        # Make predictions on the validation set
        y_pred = model.predict(X_val)
        
        # Calculate the MAE loss
        mae_loss = mean_absolute_error(y_val, y_pred)
        fold_mae_losses.append(mae_loss)

        # Calculate the RMSE loss
        rmse_loss = np.sqrt(mean_squared_error(y_val, y_pred))
        fold_rmse_losses.append(rmse_loss)
      
        # Report for the current fold
        print(f"\tFold {fold + 1}, Validation MAE Loss: {mae_loss}, RMSE Loss: {rmse_loss}")
        
    # Calculate the average and standard deviation of the losses over all folds
    avg_mae_loss = np.mean(fold_mae_losses)
    std_mae_loss = np.std(fold_mae_losses)
    avg_rmse_loss = np.mean(fold_rmse_losses)
    std_rmse_loss = np.std(fold_rmse_losses)

    # Create the report dictionary
    report_dict = {
        "Validation MAE Loss (Avg)": avg_mae_loss,
        "Validation MAE Loss (Std)": std_mae_loss,
        "Validation RMSE Loss (Avg)": avg_rmse_loss,
        "Validation RMSE Loss (Std)": std_rmse_loss,
        "Total Training Time (seconds)": end_time - start_time,
        "Best Hyperparameters": model.best_params_
    }
    
    

    save_and_report_model_artifacts(report_dict, model, model.best_params_, model_folder, model_type)

if __name__ == "__main__":
    main()


Folder MultipleRegression3 created at src/trained_models/MultipleRegression3


KFold Progress: 0it [00:00, ?it/s]

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 2/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.665 total time=   8.9s
[CV 1/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.661 total time=   9.0s
[CV 3/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.606 total time=   5.6s
[CV 1/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.661 total time=   5.8s
[CV 2/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.665 total time=   4.4s
[CV 3/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.606 total time=   4.6s
[CV 1/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.661 total time=   4.4s
[CV 2/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.665 total time=   4.5s
[CV 3/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.606 total time=   4.4s
[CV 1/3] END copy_X=True, fit_intercept=False, positive=False;, score=-4.661 total time=

KFold Progress: 1it [01:06, 66.15s/it]

	Fold 1, Validation MAE Loss: 4.670590615976926, RMSE Loss: 9.149559675514807
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.756 total time=   4.4s
[CV 2/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.672 total time=   4.5s
[CV 1/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.756 total time=   4.5s
[CV 3/3] END copy_X=True, fit_intercept=True, positive=True;, score=-4.597 total time=   4.5s
[CV 2/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.672 total time=   4.5s
[CV 3/3] END copy_X=True, fit_intercept=True, positive=False;, score=-4.597 total time=   4.5s
[CV 1/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.756 total time=   4.4s
[CV 2/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.672 total time=   4.5s
[CV 3/3] END copy_X=True, fit_intercept=False, positive=True;, score=-4.597 total time=   4.4s
[CV 1/3] E

KFold Progress: 2it [02:06, 63.09s/it]

	Fold 2, Validation MAE Loss: 4.647541698041453, RMSE Loss: 9.156946782340356


####################


Report of training results: 


{'Best Hyperparameters': {'copy_X': True,
                          'fit_intercept': True,
                          'positive': True},
 'Total Training Time (seconds)': 58.918001890182495,
 'Validation MAE Loss (Avg)': 4.659066157009189,
 'Validation MAE Loss (Std)': 0.011524458967736795,
 'Validation RMSE Loss (Avg)': 9.153253228927582,
 'Validation RMSE Loss (Std)': 0.003693553412774442}


####################



Report saved at model folder.
Filename: src/trained_models/MultipleRegression3/MultipleRegression_report3.txt.

Serialized model saved at model folder.
 Filename: src/trained_models/MultipleRegression3/MultipleRegression_model3.joblib.

Hyperparameters saved at model folder.
 Filename: src/trained_models/MultipleRegression3/MultipleRegression_hyperparameters3.json.

Hyperparameter optimization log saved at model folder.
 Filename: src/trained

<br>

We created an utility that **serializes the model**, stores a log of the **`hyperparameter` optimization process**, creates a **report of the final results in `JSON` format** and also stores **the best hyperparameters** as a `dict` and `serializes` them to a `JSON` file under the path **`src/trained_models/[MODELNAME][MODELCOUNTER]`**.

<br>

```python
save_and_report_model_artifacts(report_dict, model, model.best_params_, model_folder, model_type)
```

<br>

The relative paths are listed at the end of the training cycle, as something  this:

<br>

```bash
Report saved at model folder.
Filename: src/trained_models/MultipleRegression3/MultipleRegression_report3.txt.

Filename: src/trained_models/MultipleRegression3/MultipleRegression_hyperparameters3.json.

Hyperparameter optimization log saved at model folder.
Filename: src/trained_models/MultipleRegression3/MultipleRegression_hyperparameter_optimization_log3.json.
```

<br>

Once you run the cell above, you should get a result like this:

<br>

```python
####################


Report of training results: 


{'Best Hyperparameters': {'copy_X': True,
                          'fit_intercept': True,
                          'positive': True},
 'Total Training Time (seconds)': 58.918001890182495,
 'Validation MAE Loss (Avg)': 4.659066157009189,
 'Validation MAE Loss (Std)': 0.011524458967736795,
 'Validation RMSE Loss (Avg)': 9.153253228927582,
 'Validation RMSE Loss (Std)': 0.003693553412774442}

####################
```

<br>

This report shows:

- A Dict of the **best  hyperparameters**.

- The total training time in **seconds**.

- The average  **Mean Absolute Error Loss** on the validation runs as *percentage points*.

- The standard deviation of the **Mean Absolute Error Loss** on the validation runs as *percentage points*.

- The average **Root Mean Squared Error Loss** on the validation runs as *percentage points*.

- The standard deviation of the  **Root Mean Squared Error Loss** on the validation runs as *percentage points*.

<br>

The Standard Deviation between each validation runs show how **stable** is the model when predicting with different data. 

With this specific model we got, for the **MAE Loss Metric**, a deviation of `~0.011%` between each validation run. We consider this a sign of a stable model. Meanwhile, for the **RMSE Loss Metric** we got an **Standard Deviation** of `~0.0036%`. This shows that our model is even more stable when we **punish** large errors and outliers.

On the other hand, the average of the validation runs show us how does the model behave when confronting new data, and how does the results differenciate themselves from the actual target variable (`y_true`).

With this specific model we got, for the **MAE Loss Metric**, an average of `~4.65%` through all the validation runs. This means that our model on average differs  $\pm \sim 4.65 \% $  from the `y_true` target variable.  Meanwhile, for the **RMSE Loss Metric** we got an average of $\sim 9.15 \% $ from the `y_true` target variable . This shows that our model is being affected by the presence of  predictions with large errors, likely a result of the flattening of the `X` satellital images to accomodat the necessities of the `MultipleRegressor`, thus losing some of the underlying relations between the data channels and specially the spatial context of each pixel.

Overall, although the results of this model are surprisingly good compared to our best performing model, taking into account that it does not accomodate the nature of the data. We expect models that **preserve the spatial relations of the images** to have a better performance, as we will see later with the `VGGUdeaSpectral` neural network.

Whether `~4.65%` is a good result or not will be discussed in our next notebook.

<br>




---

<br>

## **ParzenWindowModel.py**

<br>

A Parzen Window model is essentially a non-parametric technique for estimating the probability density function of a random variable. When implemented using a `KNeighborsRegressor` (which was the approach choosen), it works by finding the 'k' nearest neighbors of a point and averaging their values to make a prediction. This is a type of instance-based learning where predictions are made for a new instance (data point) based on the instances of training data that are nearest in the input space.

<br>

For our regression problem related to Satellital Imagery, this model should not be a good fit because:

<br>

1. High Dimensionality: Satellite images are high-dimensional data, where each pixel can be a feature. KNeighborsRegressor and similar instance-based methods suffer in high-dimensional spaces because the concept of "nearest" becomes less meaningful in high-dimensional spaces.

2. Computational Intensity: Finding the nearest neighbors is computationally expensive when dealing with thousands or millions of pixels from satellite images.

3. Spatial Autocorrelation: Satellite data often exhibit spatial autocorrelation, where nearby pixels are more similar to each other than to distant pixels. This spatial information is crucial and is not effectively utilized by simple distance metrics used in nearest neighbors algorithms. 

4. Spatial Information loss: We need to flatten the images to feed them into the ParzenWindow, thus we lose the intrinsic spatial relations between the pixels and images, alongside their interpretability.

5. Non-linearity: The relationship between the spectral values in satellite images and the output variable (like vegetation coverage) is often complex and non-linear. The KNeighborsRegressor assumes that points that are near each other in feature space have similar output values, which may not always be true for the complex patterns present in satellite images.

6. Storage Requirement: Instance-based methods require storing the entire dataset in memory to make predictions, this was not a problem for us since we had enough `RAM` to load the whole dataset. But if we had trained with bigger images or more channels, we will quickly ran out of memory.

<br>

For these reasons, we do not expect good results for this model. Nonetheless, we implement it here as an academic exercise.

<br>

### *Qualities of the ParzenWindowModel.py*

- **Model Definition**: Available at [src/classes/ParzenWindowModel.py](src/classes/ParzenWindowModel.py)

- **Training Cycle**: Available at [src/generate_models/train_ParzenWindow.py](src/generate_models/train_ParzenWindow.py)

- **Compatible with**: sklearn **[BaseEstimator](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html)** and **[RegressorMixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.RegressorMixin.html)**

- **Hyperparameter Optimization SearchSpace**: 

    ```python
    param_dist = {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance', 'gaussian'],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': randint(15, 46),
        'p': [1, 2]
    }
    ```
- **Model Optimization Utility**: `RandomizedSearchCV` with scoring to *minimize MAE* and Shuffled Kfolds with `cv=3` and a 20% val split. The `RMSE` is used as a complementary metric.

    ```python
    model = RandomizedSearchCV(
                                ParzenWindowRegressor(),
                                param_distributions=param_dist,
                                n_jobs=2,            # really unstable. move back to 1 if crashing
                                n_iter=30,
                                cv=3,
                                verbose=3,
                                random_state=None,
                                scoring='neg_mean_absolute_error',  #  minimize MAE
                            )
    ```



<br>
<br>
<br>

The code provided below is a slight modification of the one available on the scripts (we changed the import structure to match the package structure)




In [1]:
##  MODEL DEFINITION

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.neighbors import KNeighborsRegressor

class ParzenWindowRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_neighbors=5, weights='uniform', algorithm='auto',
                 leaf_size=30, p=2, metric='minkowski', metric_params=None):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.algorithm = algorithm
        self.leaf_size = leaf_size
        self.p = p
        self.metric = metric
        self.metric_params = metric_params
        self.model = KNeighborsRegressor(
            n_neighbors=self.n_neighbors, 
            weights=self.weights, 
            algorithm=self.algorithm, 
            leaf_size=self.leaf_size, 
            p=self.p, 
            metric=self.metric,
            metric_params=self.metric_params
        )
        
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        return self.model.predict(X)


**WARNING**

The code below **crashes** on our `Jupyter` environments, likely due to the high requirements of memory and computing power. We will show the results obtained from the [script]([src/generate_models/train_ParzenWindow.py](src/generate_models/train_ParzenWindow.py)), but we will not execute the next cell.

The next cell will be left as an illustrative example.

<br>



In [ ]:
## TRAINING CYCLE

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import randint, uniform

import numpy as np
import torch
import time
from tqdm import tqdm

# Import utility functions
from src.utils import generate_unique_folder, save_and_report_model_artifacts

# Import custom class
from src.classes.ParzenWindowModel import ParzenWindowRegressor

def main():
    # Load the data
    X = torch.load('X_tensor.pth').numpy()
    y = torch.load('y_tensor.pth').numpy()
    
    # Flatten the images
    X = X.reshape(X.shape[0], -1)
    
    # Initialize K-Fold cross-validation # this is legacy since we implemented hyperparam_search with CV
    kf = KFold(n_splits=2, shuffle=True, random_state=None) 
    
    # Hyperparameter distr dict to search
    param_dist = {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance', 'gaussian'],
        'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': randint(15, 46),
        'p': [1, 2]
    }
    
    # Initialize the model with hyperparameter grid
    model = RandomizedSearchCV(
                                ParzenWindowRegressor(),
                                param_distributions=param_dist,
                                n_jobs=2,            # really unstable. move back to 1 if crashing
                                n_iter=30,
                                cv=3,
                                verbose=3,
                                random_state=None,
                                scoring='neg_mean_absolute_error',  #  minimize MAE
                            )

    
    # Create a folder for this specific model training
    model_type = "ParzenWindow"
    model_folder = generate_unique_folder(model_type)
    print(f"Folder {model_folder[0]} created at {model_folder[1]}")
    
    # Initialize variables for cross-validation
    fold_mae_losses = []
    fold_rmse_losses = []
    
    # Cross-validation loop
    for fold, (train_index, val_index) in enumerate(tqdm(kf.split(X), desc='KFold Progress')):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        # Train the model on the training set
        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()
        
        # Make predictions on the validation set
        y_pred = model.predict(X_val)
        
        # Calculate the MAE loss
        mae_loss = mean_absolute_error(y_val, y_pred)
        fold_mae_losses.append(mae_loss)
        
        # Calculate the RMSE loss
        rmse_loss = np.sqrt(mean_squared_error(y_val, y_pred))
        fold_rmse_losses.append(rmse_loss)
        
        # Report for the current fold
        print(f"Fold {fold + 1}, Validation MAE Loss: {mae_loss}, RMSE Loss: {rmse_loss}")
    
    # Calculate the average and standard deviation of the losses over all folds
    avg_mae_loss = np.mean(fold_mae_losses)
    std_mae_loss = np.std(fold_mae_losses)
    avg_rmse_loss = np.mean(fold_rmse_losses)
    std_rmse_loss = np.std(fold_rmse_losses)
    
    # Create the report dictionary
    report_dict = {
        "Validation MAE Loss (Avg)": avg_mae_loss,
        "Validation MAE Loss (Std)": std_mae_loss,
        "Validation RMSE Loss (Avg)": avg_rmse_loss,
        "Validation RMSE Loss (Std)": std_rmse_loss,
        "Total Training Time (seconds)": end_time - start_time,
        "Best Hyperparameters": model.best_params_,
    }
    
    save_and_report_model_artifacts(report_dict, model, model.best_params_, model_folder, model_type)

if __name__ == "__main__":
    main()

<br>

<br>

Like in our `MultipleRegressorModel.py`, we generate a series of artifacts. We store a log of the **`hyperparameter` optimization process**, create a **report of the final results in `JSON` format** and also store **the best hyperparameters** as a `dict` and `serializes` them to a `JSON` file under the path **`src/trained_models/[MODELNAME][MODELCOUNTER]`**.

<br>

At our `trained_models` directory, we fetch the `ParzenWindow0` folder and show next the results of the training.

<br>

From `src/trained_models/ParzenWindow0/ParzenWindow_report0.txt` we get:

```python
{
    "Validation MAE Loss (Avg)": 10.121114730834961,
    "Validation MAE Loss (Std)": 0.745750904083252,
    "Validation RMSE Loss (Avg)": 17.552146911621094,
    "Validation RMSE Loss (Std)": 1.4414606094360352,
    "Total Training Time (seconds)": 2867.5542833805084,
    "Best Hyperparameters": {
        "algorithm": "kd_tree",
        "leaf_size": 18,
        "metric": "minkowski",
        "n_neighbors": 10,
        "p": 2,
        "weights": "gaussian"
    }
}
```

<br>

We can observe that:

- The stability of the `ParzenWindowModel` is significantly worse than our `MultipleRegressor`, approximately **`~390`** times less stable between validation runs for the **RMSE Loss Metric** and **`64.71`** times less stable between validation runs for the **MAE Loss Metric**. 

- The results were more than twice worse for this model for the **MAE Loss Metric** compared to the `MultipleRegressorModel.py`

- The results were `0.87` times worse for this model for the **RMSE Loss Metric** compared to the `MultipleRegressorModel.py`

- The chosen distance metric was `minkowsky` distance in `p` mode 2. Which simplifies to euclidean distance.

<br>

Overall, and expectably, this model shows worse performance, stability and a complexity of over 58 times compared with the MultipleRegressor. We can conclude from this results that this modelling strategy does not accomodate well enough to the particularities of our satellital imagery data as discussed on the [ParzenWindowModel Definition](#parzenwindowmodelpy)

<br>
<br>

<br>

---

<br>

## **VGGUdeaSpectral.py**

<br>

`VGGUdeaSpectral` is a custom convolutional neural network implemented by us from the `Pytorch` dependencies to extend and harness utility from its functionalities.

We defined its custom architecture based off the *Visual Geometry Group (VGG)* proposed on the paper **Very Deep Convolutional Neural Networks for Large-Scale Image Recognition** by Simonyan and Zisserman in 2014. 

Our specific architecture uses the signature *3x3 convolutional layers* with **sequential doubling of the number of filters after each max pooling layer**

The unique additions of our architecture are:

- **Band customizability**: The traditional `VGGs` only accept three channels (for RGB). We extend over this functionality to accept any number of bands that might be fed to the network .

- **Spectral Attention**: The SpectralAttention mechanism is a type of attention mechanism that focuses on the spectral characteristics of the input data this means the mechanism pays more attention to certain  bands of the images that are more informative for the task at hand (vegetation coverage).

- **Built-in batch Normalization and Dropout**: The traditional `VGGs` do not apply these stabilization and regularization techniques by default. We include them within the architecture to handle the common  noisiness of satellital data.

- **Complexity simplification by reducing convolutiona layers and stages**: Thanks to the `SpectralAttention` mechanism, our objective is to be able to tradeoff the inclusion of the `SpectralAttention` (which we expect it will lead the network to learn more efficient data patterns)  thus allowing us to reduce the complexity of the whole network. This approach allows us to keep overfitting in check while also reducing computing costs.



<br>

### *Qualities of the VGGUdeaSpectral.py*

- **Model Definition**: Available at [src/classes/VGGUdeaSpectral.py](src/classes/VGGUdeaSpectral.py)

- **Training Cycle**: Available at [src/generate_models/train_VGGUdeaSpectral.py](src/generate_models/train_VGGUdeaSpectral.py)

- **Compatible with**: PyTorch **[NeuralNetwork base module (nn)](https://pytorch.org/docs/stable/nn.html)**

- **GPU Compatibility**: YES (if your hardware is compatible and you installed the dependencies specified at the [README.MD](README.MD))

- **Hyperparameter Optimization SearchSpace**: 

    ```python
    param_dist = {
        'module__num_filters1': [32, 64, 128],
        'module__num_filters2': [64, 128, 256],
        'module__num_filters3': [128, 256, 512],
        'module__activation_type': ['relu', 'sigmoid', 'tanh'],
        'module__dropout_rate': uniform(0.2, 0.5),
        'module__fc1_out_features': [512, 1024, 2048],
        'module__fc2_out_features': [256, 512, 1024],
        'lr': [0.01, 0.001, 0.0001],
        'max_epochs': [5, 10, 20]
    }
    ```


- **Model Optimization Utility**: We used a compatibility layer between `PyTorch` and `sklearn` called **`skorch`**.
    
    This allows us to use `RandomizedSearchCV` with scoring to *minimize MAE* and Shuffled Kfolds with `cv=3` and a 20% val split. The `RMSE` is used as a complementary metric.

    <br>

    ```python
    net = NeuralNetRegressor(
        VGGUdeaSpectral,
        module__num_bands=3,
        iterator_train__shuffle=True,
        iterator_valid__shuffle=False,
        max_epochs=10,
        lr=0.01,
        batch_size=32,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        train_split=ValidSplit(cv=0.2, stratified=False),
        callbacks=[EpochScoring(rmse_score, name='valid_rmse', lower_is_better=False)]
    )


    model = RandomizedSearchCV(
                            net,
                            param_distributions=param_dist,
                            n_iter=10,
                            cv=3,
                            verbose=3,
                            random_state=42,
                            scoring='neg_mean_absolute_error'
    )
    ```
    <br>

    Due to the increased computing cost of training this model, the `n_iter` was defined to a low number of *10 iterations* (which combined with the cross validation process still amounts to 30 different fit steps)


<br>
<br>
<br>

The code provided below is a slight modification of the one available on the scripts (we changed the import structure to match the package structure)



In [ ]:
## MODEL DEFINITION

import torch
import torch.nn as nn
import torch.nn.functional as F

from src.classes.SpectralAttention import SpectralAttention

class VGGUdeaSpectral(nn.Module):
    def __init__(self, num_bands, 
                 num_filters1=64, 
                 num_filters2=128, 
                 num_filters3=256, 
                 activation_type='relu', 
                 dropout_rate=0.5,
                 fc1_out_features=1024,
                 fc2_out_features=512,
                 number_out_features=1):
        
        super(VGGUdeaSpectral, self).__init__()
        
        # Activation function selection
        if activation_type == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation_type == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation_type == 'tanh':
            self.activation = nn.Tanh()
        else:
            raise ValueError("Invalid activation_type")
        
        self.conv_stage1 = nn.Sequential(
            nn.Conv2d(num_bands, num_filters1, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters1),
            SpectralAttention(num_filters1),
            self.activation,
            nn.Conv2d(num_filters1, num_filters1, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters1),
            SpectralAttention(num_filters1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_stage2 = nn.Sequential(
            nn.Conv2d(num_filters1, num_filters2, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters2),
            SpectralAttention(num_filters2),
            self.activation,
            nn.Conv2d(num_filters2, num_filters2, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters2),
            SpectralAttention(num_filters2),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_stage3 = nn.Sequential(
            nn.Conv2d(num_filters2, num_filters3, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters3),
            SpectralAttention(num_filters3),
            self.activation,
            nn.Conv2d(num_filters3, num_filters3, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters3),
            SpectralAttention(num_filters3),
            self.activation,
            nn.Conv2d(num_filters3, num_filters3, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_filters3),
            SpectralAttention(num_filters3),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((12, 12))
        
        self.fc1 = nn.Sequential(
            nn.Linear(num_filters3 * 12 * 12, fc1_out_features),
            nn.BatchNorm1d(fc1_out_features),
            self.activation,
            nn.Dropout(dropout_rate)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(fc1_out_features, fc2_out_features),
            nn.BatchNorm1d(fc2_out_features),
            self.activation,
            nn.Dropout(dropout_rate)
        )
        
        self.output = nn.Linear(fc2_out_features, number_out_features)
        
    def forward(self, x):
        x = self.conv_stage1(x)
        x = self.conv_stage2(x)
        x = self.conv_stage3(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.output(x)
        return x


**WARNING**

The code below **crashes** on our `Jupyter` environments, likely due to the high requirements of memory and computing power. We will show the results obtained from the [script](src/generate_models/train_VGGUdeaSpectral.py), but we will not execute the next cell.

The next cell will be left as an illustrative example.

<br>



In [ ]:
## TRAINING CYCLE

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import RandomizedSearchCV
from skorch import NeuralNetRegressor
from skorch.dataset import ValidSplit
from skorch.callbacks import EpochScoring
import numpy as np
from scipy.stats import uniform
from sklearn.metrics import mean_squared_error
import time

# Custom utility functions
from src.utils import generate_unique_folder, save_and_report_model_artifacts

# Import the custom VGGUdeaSpectral class
from src.classes.VGGUdeaSpectral import VGGUdeaSpectral

def rmse_score(net, X, y):
    y_pred = net.predict(X)
    rmse = (mean_squared_error(y_true=y, y_pred=y_pred)) ** 0.5
    return -rmse  # Skorch tries to maximize the score, so negate the RMSE

def main():
    # Load data
    X = torch.load('X_tensor.pth')
    y = torch.load('y_tensor.pth').view(-1,1)

    # Convert X to float32
    X = X.to(dtype=torch.float32)

    # Create DataLoader
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Initialize NeuralNetRegressor
    net = NeuralNetRegressor(
        VGGUdeaSpectral,
        module__num_bands=3,
        iterator_train__shuffle=True,
        iterator_valid__shuffle=False,
        max_epochs=10,
        lr=0.01,
        batch_size=32,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        train_split=ValidSplit(cv=0.2, stratified=False),
        callbacks=[EpochScoring(rmse_score, name='valid_rmse', lower_is_better=False)]
    )

    param_dist = {
        'module__num_filters1': [32, 64, 128],
        'module__num_filters2': [64, 128, 256],
        'module__num_filters3': [128, 256, 512],
        'module__activation_type': ['relu', 'sigmoid', 'tanh'],
        'module__dropout_rate': uniform(0.2, 0.5),
        'module__fc1_out_features': [512, 1024, 2048],
        'module__fc2_out_features': [256, 512, 1024],
        'lr': [0.01, 0.001, 0.0001],
        'max_epochs': [5, 10, 20]
    }

    model = RandomizedSearchCV(net, param_distributions=param_dist, n_iter=10, cv=3, verbose=3, random_state=42, scoring='neg_mean_absolute_error')

    # Train the model
    start_time = time.time()
    model.fit(X, y)
    end_time = time.time()

    # Create the report dictionary
    report_dict = {
        "Validation MAE Loss (Best)": -model.best_score_,
        "Validation RMSE (Best)": -rmse_score(model, X, y),
        "Total Training Time (seconds)": end_time - start_time,
        "Best Hyperparameters": model.best_params_,
    }

    # Create a folder for this specific model training
    model_type = "VGGUdeaSpectral"
    model_folder = generate_unique_folder(model_type)

    # Save model and report
    save_and_report_model_artifacts(report_dict, model, model.best_params_, model_folder, model_type)

if __name__ == "__main__":
    main()


<br>

<br>

Like in our `MultipleRegressorModel.py` and `ParzenWindow.py`, we generate a series of artifacts. We store a log of the **`hyperparameter` optimization process**, create a **report of the final results in `JSON` format** and also store **the best hyperparameters** as a `dict` and `serializes` them to a `JSON` file under the path **`src/trained_models/[MODELNAME][MODELCOUNTER]`**.

<br>

At our `trained_models` directory, we fetch the `VGGUdeaSpectral0` folder and show next the results of the training.

<br>

From `src/trained_models/VGGUdeaSpectral0/VGGUdeaSpectral_report0.txt` we get:

```python
{
    "Validation MAE Loss": 3.799692153930664,
    "Validation RMSE Loss": 6.044155090705238,
    "Total Training Time (seconds)": 28129.791773557663,
    "Best Hyperparameters": {
        "lr": 0.0001,
        "max_epochs": 20,
        "module__activation_type": "tanh",
        "module__dropout_rate": 0.6330880728874676,
        "module__fc1_out_features": 2048,
        "module__fc2_out_features": 512,
        "module__num_filters1": 32,
        "module__num_filters2": 128,
        "module__num_filters3": 256
    }
}
```


<br>



<br>

---

## **CONCLUSIONS DERIVED FROM THE RESULTS OF THE MODELS**

<br>


From these results, we can observe that:

- Even with the added raw power of the `GPU` training, this model takes  `~ x488.85`  more time to train than the `MultipleRegressionModel` and `~ x9.8` more time to train than the `ParzenWindowModel`. The computational cost is evident.

- On average, each fit on the  `VGGUdeaSpectral` convolutional neural network takes `~15 minutes` with our equipment. (64 DDR4 RAM 2999MHz / Ryzen 7 5800x / RX7900 XT on Ubuntu 21.03 LTS with the dependencies specified on the [requirements.txt](requirements.txt))

- Our MAE Loss improved from  `~4.65` on our `MultipleRegressionModel` to `~3.8` on our `VGGUdeaSpectral` neural network.

- Our RMSE Loss improved from `~9.15` on our `MultipleRegressionModel` to `~6.04` on our `VGGUdeSpectral` neural network. This means that our neural network is significantly more robust when dealing with predictions related to outliers and that the errors are more close to the target!

- Our search on the *parameter space* was extremely shallow. We only tried 10 different combinations of parameters, if we were to do a `grid_search` the total number of combinations would be `19683`. 


So, we can summarize our results with:

1. The best performance comes from the `VGGUdeaSpectral` model.

2. The most robust (the one most resistant to outliers and most consistent with predictions near-target) is the `VGGUdeaSpectral` model.

3. **Our Specific** implementation of the  `ParzenWindow` is more computational complex, less stable and with the worse performance when compared with the `VGGUdeaSpectral` model and the `MultipleRegressor` model.

4. Our `VGGUdeaSpectral` model takes `~ x488.85`  more time expensive to train than our `MultipleRegressor` model. But offers an improvement of `~18.28%` on the MAE Loss metric and a `~33.99%` improvement on the RMSE Loss Metric.

5. The **parametric space** of the `VGGUdeaSpectral` model was only shallowly explored.

<br>

---

<br>

## FUTURE WORK AND USES FOR THE MODEL

---

<br>

### Future Work

<br>

To further improve on the model, we recommend the following work:

1. Expand the hyperparameter search optimization. We made an estimation based on our avg training time for each fit performed.

This table shows an estimation of the minutes, hours and days that it would take those `n_iterations` :

| n_iterations | Total Training Time (Minutes) | Total Training Time (Hours) | Total Training Time (Days) |
|--------------|------------------------------|-----------------------------|----------------------------|
| 1            | 45.0                         | 0.75                        | 0.03125                    |
| 2            | 90.0                         | 1.5                         | 0.0625                     |
| 3            | 135.0                        | 2.25                        | 0.09375                    |
| 4            | 180.0                        | 3.0                         | 0.125                      |
| 5            | 225.0                        | 3.75                        | 0.15625                    |
| 6            | 270.0                        | 4.5                         | 0.1875                     |
| 7            | 315.0                        | 5.25                        | 0.21875                    |
| 8            | 360.0                        | 6.0                         | 0.25                       |
| 9            | 405.0                        | 6.75                        | 0.28125                    |
| 10           | 450.0                        | 7.5                         | 0.3125                     |
| 20           | 900.0                        | 15.0                        | 0.625                      |
| 30           | 1350.0                       | 22.5                        | 0.9375                     |
| 40           | 1800.0                       | 30.0                        | 1.25                       |
| 50           | 2250.0                       | 37.5                        | 1.5625                     |
| 100          | 4500.0                       | 75.0                        | 3.125                      |
| 200          | 9000.0                       | 150.0                       | 6.25                       |
| 300          | 13500.0                      | 225.0                       | 9.375                      |
| 400          | 18000.0                      | 300.0                       | 12.5                       |
| 500          | 22500.0                      | 375.0                       | 15.625                     |
| 1000         | 45000.0                      | 750.0                       | 31.25                      |
| 2000         | 90000.0                      | 1500.0                      | 62.5                       |
| 3000         | 135000.0                     | 2250.0                      | 93.75                      |
| 4000         | 180000.0                     | 3000.0                      | 125                        |
| 5000         | 225000.0                     | 3750.0                      | 156.25                     |
| 6000         | 270000.0                     | 4500.0                      | 187.5                      |
| 7000         | 315000.0                     | 5250.0                      | 218.75                     |
| 8000         | 360000.0                     | 6000.0                      | 250                        |
| 9000         | 405000.0                     | 6750.0                      | 281.25                     |
| 10000        | 450000.0                     | 7500.0                      | 312.5                      |

<br>

Perhaps *100 different combinations* (3 days) would be a good start.


2. Try out more bands! Our `VGGUdeaSpectral` accepts more than 3 bands. This would require an analysis process to select the best candidates to add.

3. To test the **generalization power of the model even further**, we recommend to test it  with even more new data!. A good start would be to test a different year or images from a different `Instrument`; if the results are good a natural step would be to test on a different geographical **Region of Interest** (`ROI`)

4. Explore the usage of  more advanced techniques like Bayesian Optimization to optimize the hyperparameters.

5. Explore the performance of the model when receiving images of higher dimensions. The training was done with *100x100* images, how would the model behave with images of higher resolution?

6. Explore the impact on computational resources when training the model with higher dimensional images. Will it make the `VGGUdeaSpectral` not feasible to be trained anymore?

<br>

---


<br>

### **USES FOR THE MODEL**

<br>

We recognize that our knowledge on the subjects of *ecology* and *geology* are limited, and for us to define **performance thresholds** we will require the  advice of experts on the subjects.

However, based on a review of current literature we find feasible to implement this model (with proper validation with field experts) for applications such as:

1. **Rapid assessment of endangered ecosystems.** It has been found that the usage of *Satellital Imagery* is feasible to monitor the changes in **Mangrobe Forests** [1]. With the help of field experts, perhaps the same approach could be applied to the Antioquian Andean Forests and tropical rainforests.

2. **Land use analysis**. Loss of vegetation is related to land development or illegal exploitations [2]. We find worthy of analysis if by systematically analyzing areas with non-progressive loss of vegetation we could detect illegal logging or mining operations in Antioquia.

3. **Natural Disaster Prediction**. The loss of vegetation alongside geographical landmarks or pluvial sources has been related to landslides and floods [3]. Remote sensing techniques based on Satellital Imagery are already being used in Asia[4], perhaps our model would be a good tool to systematically analyse specific features alongside a model that is capable of identifying frontiers and borders of rivers and other ground/rainwater resources.



<br>
<br>
<br>


#### **REFERENCES**


[1] C. -H. Shih, P. -C. Chen, Y. -J. Shih, T. -J. Chu and Y. -M. Lu, "Feasibility for Rapid Assessment of Mangroves by the Application of Satellite Imagery Technique," 2019 IEEE International Conference on Architecture, Construction, Environment and Hydraulics (ICACEH), Xiamen, China, 2019, pp. 44-46, doi: 10.1109/ICACEH48424.2019.9042101.

[2] Yumin Tan, Bingxin Bai and M. S. Mohammad, "Time series remote sensing based dynamic monitoring of land use and land cover change," 2016 4th International Workshop on Earth Observation and Remote Sensing Applications (EORSA), Guangzhou, China, 2016, pp. 202-206, doi: 10.1109/EORSA.2016.7552797.

[3] J. Sun et al., "Regional-Scale Monitoring of Rice Flood Disaster Based on Multi-Temporal Remote Sensing Images," 2018 7th International Conference on Agro-geoinformatics (Agro-geoinformatics), Hangzhou, China, 2018, pp. 1-4, doi: 10.1109/Agro-Geoinformatics.2018.8476147.

[4] M. Kalidhas and R. Sivakumar, "Image processing and Supervised Classification of LANDSAT data for Flood Impact Assessment on Land Use and Land Cover," 2022 2nd International Conference on Technological Advancements in Computational Sciences (ICTACS), Tashkent, Uzbekistan, 2022, pp. 437-440, doi: 10.1109/ICTACS56270.2022.9988164.

<br>

<br>

---